In [12]:
from os.path import expanduser, join
import pathlib

root = pathlib.Path().resolve()

In [13]:
from os import listdir

dir = listdir(root)
if 't.ex-Graph' in dir:
  root = join(root, 't.ex-Graph')

In [14]:
import sys
sys.path.insert(1, join(root, 'lib'))

import config
import data
import model
import export

In [15]:
datasets = [{
    'label': 'HTTP/S Graph (SLDs)',
    'data': data.read(join(root, 'data', 'graph-data-sld.csv')).sample(n=200)
  }, {
    'label': 'HTTP/S Graph (FQDN)',
    'data': data.read(join(root, 'data', 'graph-data-fqdn.csv')).sample(n=200)
  }
]

In [16]:
for dataset in datasets:
  data.binary_classification_labels('binary_tracker', dataset.get('data'))

In [17]:
for dataset in datasets:
  data.multi_classification_labels('multi_tracker', dataset.get('data'))

In [18]:
features = data.feature_vector(datasets[0].get('data'))

In [19]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

models = {
  'continuous': [
    LinearRegression(n_jobs=-1),
    RandomForestRegressor(n_estimators=200, random_state=0, n_jobs=-1)
  ],
  'category': [
    LogisticRegression(solver='lbfgs', max_iter=1000, n_jobs=-1),
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianNB(),
    DecisionTreeClassifier(),
    RandomForestClassifier(max_depth=2, random_state=0),
    AdaBoostClassifier(),
    QuadraticDiscriminantAnalysis(),
    MLPClassifier(alpha=1, max_iter=1000)
  ]
}

In [20]:
%matplotlib agg

results = model.compute_results(
  datasets, 
  models, 
  features, 
  [
    # 'weight', 
    'binary_tracker', 
    # 'multi_tracker'
  ]
)

In [ ]:
export.classification_results(results, root)

In [ ]:
export.aggregated_classification_results(root)

In [ ]:
ncols = max(len(models['continuous']), len(models['category']))
export.feature_importances(2, ncols, results, root)

FileNotFoundError: [Errno 2] No such file or directory: '/home/philip/t.ex-Graph/results/feature-importances-HTTP/S Graph (SLDs).pdf'